In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import time
import os

2024-11-27 11:40:35.072407: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 11:40:35.405163: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 11:40:36.179035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
BASE_DIR = "/home/aishwaryakunche/DL project"
TRAIN_DIR = os.path.join(BASE_DIR, "training")
TEST_DIR = os.path.join(BASE_DIR, "testing")

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 150, 150

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='sparse'
)


Found 2870 images belonging to 4 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='sparse'
)

Found 394 images belonging to 4 classes.


In [ ]:
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for _ in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                model.add(Dropout(0.33))

            model.add(Dense(4))  # 4 classes present in the dataset: glioma, meningioma, no_tumor, pituitary
            model.add(Activation('softmax'))

            model.compile(
                loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy']
            )

            # Train the model
            model.fit(
                train_generator,
                epochs=20,
                validation_data=test_generator,
                callbacks=[tensorboard, es]
            )

            # Evaluate the model
            scores = model.evaluate(test_generator, verbose=1)
            print('Test loss:', scores[0])
            print('Test accuracy:', scores[1])

            # Save the model
            model.save(f"{NAME}-model-{scores[1]:.2f}-accuracy.h5")

            # Precision, recall, F1-score
            y_pred = model.predict(test_generator, verbose=1)
            y_pred_bool = np.argmax(y_pred, axis=1)
            print(classification_report(test_generator.classes, y_pred_bool, target_names=test_generator.class_indices.keys()))


1-conv-32-nodes-0-dense-1732687991
Epoch 1/20


/home/aishwaryakunche/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/aishwaryakunche/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3786 - loss: 1.8565

/home/aishwaryakunche/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


90/90 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - accuracy: 0.3800 - loss: 1.8502 - val_accuracy: 0.5178 - val_loss: 1.2510
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.7837 - loss: 0.5378 - val_accuracy: 0.6751 - val_loss: 1.2236
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.9230 - loss: 0.2827 - val_accuracy: 0.7310 - val_loss: 1.1727
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.9712 - loss: 0.1342 - val_accuracy: 0.7766 - val_loss: 1.3019
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.9917 - loss: 0.0652 - val_accuracy: 0.7563 - val_loss: 1.3840
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9968 - loss: 0.0402 - val_accuracy: 0.7589 - val_loss: 1.6361
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.9994 - loss: 0.0224 - val_accuracy: 0.7843 - val_loss: 1.7950
Epoch 8/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.9989 - loss: 0.0146 - val_accuracy: 0.7487 - val_loss: 2.

...

90/90 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.3138 - loss: 1.6224 - val_accuracy: 0.4010 - val_loss: 1.3452
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.6026 - loss: 0.9105 - val_accuracy: 0.4315 - val_loss: 1.6933
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.7018 - loss: 0.7085 - val_accuracy: 0.5406 - val_loss: 1.5694
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.7679 - loss: 0.5845 - val_accuracy: 0.6091 - val_loss: 1.5069
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 116s 1s/step - accuracy: 0.8160 - loss: 0.4686 - val_accuracy: 0.6269 - val_loss: 1.6350
Epoch 5: early stopping
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 410ms/step - accuracy: 0.6200 - loss: 1.6766


Test loss: 1.6350417137145996
Test accuracy: 0.6269035339355469
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 337ms/step
              precision    recall  f1-score   support

      glioma       0.33      0.09      0.14       100
  meningioma       0.30      0.35      0.32       115
    no_tumor       0.27      0.49      0.34       105
   pituitary       0.15      0.08      0.10        74

    accuracy                           0.27       394
   macro avg       0.26      0.25      0.23       394
weighted avg       0.27      0.27      0.24       394

3-conv-128-nodes-2-dense-1732693572


/home/aishwaryakunche/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - accuracy: 0.2946 - loss: 1.3968 - val_accuracy: 0.4061 - val_loss: 1.3706
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.4760 - loss: 1.1655 - val_accuracy: 0.4188 - val_loss: 1.4917
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 89s 993ms/step - accuracy: 0.6791 - loss: 0.7796 - val_accuracy: 0.4340 - val_loss: 1.5192
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 90s 999ms/step - accuracy: 0.7055 - loss: 0.6873 - val_accuracy: 0.4670 - val_loss: 1.9151
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 90s 994ms/step - accuracy: 0.7768 - loss: 0.5509 - val_accuracy: 0.4975 - val_loss: 1.4455
Epoch 5: early stopping
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 205ms/step - accuracy: 0.4591 - loss: 1.6121


Test loss: 1.445491075515747
Test accuracy: 0.4974619150161743
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 209ms/step
              precision    recall  f1-score   support

      glioma       0.24      0.08      0.12       100
  meningioma       0.17      0.07      0.10       115
    no_tumor       0.26      0.56      0.36       105
   pituitary       0.21      0.26      0.23        74

    accuracy                           0.24       394
   macro avg       0.22      0.24      0.20       394
weighted avg       0.22      0.24      0.20       394

